<div id="singlestore-header" style="display: flex; background-color: rgba(235, 249, 245, 0.25); padding: 5px;">
    <div id="icon-image" style="width: 90px; height: 90px;">
        <img width="100%" height="100%" src="https://raw.githubusercontent.com/singlestore-labs/spaces-notebooks/master/common/images/header-icons/browser.png" />
    </div>
    <div id="text" style="padding: 5px; margin-left: 10px;">
        <div id="badge" style="display: inline-block; background-color: rgba(0, 0, 0, 0.15); border-radius: 4px; padding: 4px 8px; align-items: center; margin-top: 6px; margin-bottom: -2px; font-size: 80%">SingleStore Notebooks</div>
        <h1 style="font-weight: 500; margin: 8px 0 0 4px;">Upgrade Your Virtual Workspace</h1>
    </div>
</div>

<div class="alert alert-block alert-warning">
    <b class="fa fa-solid fa-exclamation-circle"></b>
    <div>
        <p><b>Note</b></p>
        <p>This notebook can be run on a Free Starter Workspace. To create a Free Starter Workspace navigate to <tt>Start</tt> using the left nav. You can also use your existing Standard or Premium workspace with this Notebook.</p>
    </div>
</div>

<div class="alert alert-block alert-warning">
    <b class="fa fa-solid fa-exclamation-circle"></b>
    <div>
        <p><b>Note</b></p>
        <p>This notebook should be run on a shared deployment in an Organization whose plan allow for the creation of a new dedicated deployment.</p>
    </div>
</div>

This notebook will show you how to upgrade your shared deployment to a new dedicated deployment.
All the fields are ready with default values, but we encurage customize them.
The required information is:
- a new name for your new Workspace in the dedicated deployment
- the desired size of your new Workspace in the dedicated deployment (see: https://www.singlestore.com/cloud-pricing/)
- a custom name for your new workspace group
- the id of the region to create your new workspace group

<div class="alert alert-block alert-warning">
    <b class="fa fa-solid fa-exclamation-circle"></b>
    <div>
        <p><b>Warning</b></p>
        <p>During the Upgrade process data writen to the shared deployment may not be moved to the new Workspace in the dedicated deployment.</p>
        <p>The shared deployment will remain available during (and after) the upgrade process, however any writes to the shared deployment after the upgrade process begins may not be reflected on the new workspace.</p>
        <p>Please account for this behaviour when upgrading.</p>
    </div>
</div>

## Choose a Region for the new workspace group
Please select the Region you desire your new Workspace Group to be created on from the dropdown after the next cell.

In [1]:
import requests
from ipywidgets import widgets

region_response = requests.get('https://api.singlestore.com/v1/regions', headers={'Authorization': 'Bearer {}'.format(connection_password)})
regions = [[f"{r['provider']}: {r['region']}", r['regionID']] for r in region_response.json()]
region_dropdown = widgets.Dropdown(
    options = regions,
    value = regions[0][1],
    description = 'Region:',
    disabled = False,
)

display(region_dropdown)

## Choose a Name for the new workspace group
Please insert the name for your new Workspace Group in text box after the next cell.

In [2]:
from ipywidgets import widgets

workspace_group_name_text_box = widgets.Text(
    value='Upgrade Group',
    placeholder='Insert a name',
    description='Name:'
)

display(workspace_group_name_text_box)

## Choose a Name for the new workspace
Please insert the name for your new Workspace in text box after the next cell.

In [3]:
from ipywidgets import widgets

workspace_name_text_box = widgets.Text(
    value='Upgraded Workspace',
    placeholder='Insert a name',
    description='Name:'
)

display(workspace_name_text_box)

## Choose the size for the new workspace
Please insert the size for your new Workspace in text box after the next cell.
If you are unsure which size to choose, visit the [docs](https://www.singlestore.com/cloud-pricing/).

In [4]:
from ipywidgets import widgets

size_text_box = widgets.Text(
    value='S-00',
    placeholder='Insert a size',
    description='Size:',
    layout={'width': 'max-content'}
)

display(size_text_box)

<div class="alert alert-block alert-warning">
    <b class="fa fa-solid fa-exclamation-circle"></b>
    <div>
        <p><b>Warning</b></p>
        <p>During the Upgrade process data writen to the shared deployment may not be moved to the new Workspace in the dedicated deployment.</p>
        <p>The shared deployment will remain available during (and after) the upgrade process, however any writes to the shared deployment after the upgrade process begins may not be reflected on the new workspace.</p>
        <p>Please account for this behaviour when upgrading.</p>
    </div>
</div>

## Begin the Upgrade Process
Runing the next cell will start the upgrade process

In [5]:
import requests, os
from ipywidgets import widgets

workspace_id = os.environ['SINGLESTOREDB_VIRTUAL_WORKSPACE']
upgrade_response = requests.post('https://api.singlestore.com/v1/sharedtier/virtualWorkspaces/{}/upgrade'.format(workspace_id), headers={'Authorization': 'Bearer {}'.format(connection_password)}, json={
  "destinationWorkspace": {
    "name": workspace_name_text_box.value,
    "regionID": region_dropdown.value,
    "size": size_text_box.value,
    "workspaceGroupName": workspace_group_name_text_box.value
  }
})

if upgrade_response.status_code == 200:
    display(widgets.HTML(f"<div class='alert alert-block alert-success'><b class='fa fa-solid fa-check-circle'></b><div><p><b>Upgrade Started</b></p><p>See next cell for progress</p></div></div>"))
else:
    display(widgets.HTML(f"<div class='alert alert-block alert-danger'><b class='fa fa-solid fa-exclamation-triangle'></b><div><p><b>Failed to Start Upgrade</b></p><p>{upgrade_response.status_code} - {upgrade_response.text}</p></div></div>"))

## Check the progress of the Upgrade:
Running the next cell will give you progress on the upgrade process

In [6]:
import requests, os, time
from ipywidgets import widgets

workspace_id = os.environ['SINGLESTOREDB_VIRTUAL_WORKSPACE']
organization_id = os.environ['SINGLESTOREDB_ORGANIZATION']
status_response = requests.get('https://api.singlestore.com/v1/sharedtier/virtualWorkspaces/{}/upgrade'.format(workspace_id), headers={'Authorization': 'Bearer {}'.format(connection_password)})

while status_response.json()["upgradeStatus"] not in ["Completed", "Failed"]:
    display(widgets.IntProgress(
        value=["Failed", "WaitingForWorkspaceGroup", "MigratingData", "Completed"].index(status_response.json()["upgradeStatus"]),
        min=0,
        max=3,
        description='Loading:',
        orientation='horizontal'
    ))
    time.sleep(15)
    status_response = requests.get('https://api.singlestore.com/v1/sharedtier/virtualWorkspaces/{}/upgrade'.format(workspace_id), headers={'Authorization': 'Bearer {}'.format(connection_password)})


if status_response.json()["upgradeStatus"] == "Completed":
    display(widgets.HTML('<div id="singlestore-header" style="display: flex; background-color: rgba(235, 249, 245, 0.25); padding: 5px;"><a href="https://portal.singlestore.com/organizations/{}/virtual-workspaces/{}" target="_blank">New Workspace</a>'.format(organization_id, workspace_id)))
else:
    display(widgets.HTML("<div class='alert alert-block alert-danger'><b class='fa fa-solid fa-exclamation-triangle'></b><div><p><b>Failed to Complete Upgrade</b></p><p>The upgrade process failed, please contact support</p></div></div>"))

<div id="singlestore-footer" style="background-color: rgba(194, 193, 199, 0.25); height:2px; margin-bottom:10px"></div>
<div><img src="https://raw.githubusercontent.com/singlestore-labs/spaces-notebooks/master/common/images/singlestore-logo-grey.png" style="padding: 0px; margin: 0px; height: 24px"/></div>